In [ ]:
# !pip3 install geojson
# !pip3 install dash
# !pip3 install jupyter-dash
import pandas as pd
import numpy as np
import csv
import plotly.express as px
import geopandas as gpd
import json
import requests
import os
import re
%matplotlib inline
from shapely.geometry import Point
from geopandas import datasets, GeoDataFrame, read_file
from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, Input, Output
import plotly.io as pio
import plotly.express as px
from dash.dependencies import Input, Output
from dash import dash_table
pio.renderers.default = 'iframe'
workdir = os.getcwd() + '/dash_test_data/' # for Tina
# workdir = '/content/drive/MyDrive/Hazus Data/'

In [ ]:
path = f'{workdir}county_files.csv'
county_file = pd.read_csv(path, index_col = 0)
county_file["FIPS"] = county_file["FIPS"].astype(str).str.rjust(5,"0")
county_file["County"] = county_file["County"].str.replace(" ","_")

In [ ]:
county_file

,FIPS,County,Abbreviation,State
1,01001,Autauga_County,AL,Alabama
2,01003,Baldwin_County,AL,Alabama
3,01005,Barbour_County,AL,Alabama
4,01007,Bibb_County,AL,Alabama
5,01009,Blount_County,AL,Alabama
...,...,...,...,...
3230,72145,Vega_Baja_Municipio,PR,Puerto Rico
3231,72147,Vieques_Municipio,PR,Puerto Rico
3232,72149,Villalba_Municipio,PR,Puerto Rico
3233,72151,Yabucoa_Municipio,PR,Puerto Rico


In [ ]:
def set_multiindex(table,group):
    MI = pd.MultiIndex.from_product([[group],table.columns], names=['group','feature'])
    table.columns = MI
    return table

In [ ]:
state = 'DC'
fips_code = '11010'
county_name = 'District_of_Columnbia'
path = f'{workdir}/{state}/DC_11001_District_of_Columbia.zip'
# path = '/Users/tina/Desktop/dash_test_data/DC/DC_11001_District_of_Columbia.zip'
test = f'{workdir}DC/DC_11001_District_of_Columbia.zip'
example = gpd.read_file(test)
example["INTPTLON10"] = example["INTPTLON10"].astype(float)
example["INTPTLAT10"] = example["INTPTLAT10"].astype(float)
example["CensusBlock"] = example["GEOID10"].astype(int)
example = example.drop("GEOID10", axis = 1)
example = example.set_index("CensusBlock")
example = set_multiindex(example,"Location")
example

group            Location                                                     \
feature         STATEFP10 COUNTYFP10 TRACTCE10 BLOCKCE10      NAME10 MTFCC10   
CensusBlock                                                                    
110010104001019        11        001    010400      1019  Block 1019   G5040   
110010073011041        11        001    007301      1041  Block 1041   G5040   
110010098031002        11        001    009803      1002  Block 1002   G5040   
110010098033004        11        001    009803      3004  Block 3004   G5040   
110010098071009        11        001    009807      1009  Block 1009   G5040   
...                   ...        ...       ...       ...         ...     ...   
110010015004013        11        001    001500      4013  Block 4013   G5040   
110010015004005        11        001    001500      4005  Block 4005   G5040   
110010015001012        11        001    001500      1012  Block 1012   G5040   
110010013012013        11        001    001301      2013  Block 2013   G5040   
110010001004004        11        001    000100      4004  Block 4004   G5040   

group                                                                       \
feature         UR10 UACE10 UATYP10 FUNCSTAT10 ALAND10 AWATER10 INTPTLAT10   
CensusBlock                                                                  
110010104001019    U  92242       U          S   37411        0  38.842469   
110010073011041    U  92242       U          S   10006        0  38.838679   
110010098031002    U  92242       U          S   17987        0  38.838723   
110010098033004    U  92242       U          S   26637        0  38.833719   
110010098071009    U  92242       U          S   17842        0  38.832129   
...              ...    ...     ...        ...     ...      ...        ...   
110010015004013    U  92242       U          S   27891        0  38.975498   
110010015004005    U  92242       U          S   39255        0  38.977308   
110010015001012    U  92242       U          S   74332        0  38.979354   
110010013012013    U  92242       U          S   17385        0  38.955210   
110010001004004    U  92242       U          S   13148        0  38.906357   

group                                                                          
feature         INTPTLON10                                           geometry  
CensusBlock                                                                    
110010104001019 -77.008317  POLYGON ((-77.00815 38.84509, -77.00754 38.845...  
110010073011041 -77.011348  POLYGON ((-77.01016 38.83866, -77.01042 38.838...  
110010098031002 -77.004923  POLYGON ((-77.00445 38.84009, -77.00444 38.840...  
110010098033004 -77.006591  POLYGON ((-77.00589 38.83445, -77.00588 38.834...  
110010098071009 -77.008705  POLYGON ((-77.00880 38.83287, -77.00876 38.832...  
...                    ...                                                ...  
110010015004013 -77.064208  POLYGON ((-77.06312 38.97500, -77.06313 38.975...  
110010015004005 -77.062374  POLYGON ((-77.05963 38.97779, -77.05964 38.977...  
110010015001012 -77.059154  POLYGON ((-77.06305 38.97850, -77.06276 38.978...  
110010013012013 -77.066634  POLYGON ((-77.06565 38.95461, -77.06571 38.954...  
110010001004004 -77.062168  POLYGON ((-77.06121 38.90632, -77.06184 38.906...  

[6507 rows x 15 columns]

In [ ]:
path = f'{workdir}/{state}/DC_11001_District_of_Columbia.csv'
dc = pd.read_csv(path, index_col = 0, header = [0,1])
dc

group           BuildingCount                                                \
feature                  RES1 RES2 RES3A RES3B RES3C RES3D RES3E RES3F RES4   
CensusBlock                                                                   
110010001001000          73.0  0.0   7.0   4.0   1.0   2.0   1.0   1.0  0.0   
110010001001001           5.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   
110010001001002          25.0  0.0   2.0   1.0   0.0   1.0   0.0   0.0  0.0   
110010001001003          29.0  0.0   3.0   2.0   0.0   1.0   0.0   1.0  0.0   
110010001001004          20.0  0.0   2.0   1.0   0.0   0.0   0.0   0.0  0.0   
...                       ...  ...   ...   ...   ...   ...   ...   ...  ...   
110010111003050           0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   
110010111003051           0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   
110010111003052           0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   
110010111003054           0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   
110010111003057           0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   

group                 ... TreeParameter                                    \
feature         RES5  ...  StemsPerAcre TreeHeightLess40 TreeHeight40To60   
CensusBlock           ...                                                   
110010001001000  1.0  ...            82             0.23             0.31   
110010001001001  0.0  ...            51             0.23             0.31   
110010001001002  0.0  ...            31             0.23             0.31   
110010001001003  0.0  ...            30             0.23             0.31   
110010001001004  0.0  ...            50             0.23             0.31   
...              ...  ...           ...              ...              ...   
110010111003050  0.0  ...            59             0.23             0.31   
110010111003051  0.0  ...           121             0.23             0.31   
110010111003052  0.0  ...            14             0.23             0.31   
110010111003054  0.0  ...            10             0.23             0.31   
110010111003057  0.0  ...            15             0.23             0.31   

group                                                            Hurricane  \
feature         TreeHeightGreater60 TreeCollectionFactor  huBldgSchemeName   
CensusBlock                                                                  
110010001001000                0.46              0.38834  Northeast_Inland   
110010001001001                0.46              1.00000  Northeast_Inland   
110010001001002                0.46              1.00000  Northeast_Inland   
110010001001003                0.46              1.00000  Northeast_Inland   
110010001001004                0.46              1.00000  Northeast_Inland   
...                             ...                  ...               ...   
110010111003050                0.46              0.08896  Northeast_Inland   
110010111003051                0.46              1.00000  Northeast_Inland   
110010111003052                0.46              0.07741  Northeast_Inland   
110010111003054                0.46              0.20424  Northeast_Inland   
110010111003057                0.46              0.40938  Northeast_Inland   

group                                                                          
feature         huOccMapSchemeName DistToCoast SURFACEROUGHNESS WindGridIndex  
CensusBlock                                                                    
110010001001000   Northeast_Inland   30.000000            0.607             1  
110010001001001   Northeast_Inland   30.000000            0.575             1  
110010001001002   Northeast_Inland   30.000000            0.585             1  
110010001001003   Northeast_Inland   30.000000            0.582             1  
110010001001004   Northeast_Inland   30.000000            0.540             1  
...                            ...         ...              ...           ...  
1

In [ ]:
aggregated_table = dc.merge(example, how = "inner", left_index = True, right_index = True)

In [ ]:
aggregated_table_gpd = gpd.GeoDataFrame(aggregated_table, geometry =('Location','geometry'))

In [ ]:
# fig = px.choropleth_mapbox(
#     aggregated_table_gpd,
#     geojson=aggregated_table_gpd[('Location','geometry')],
#     locations=aggregated_table_gpd.index,
#     center={"lat": aggregated_table_gpd[('Location',"INTPTLAT10")].iat[0], "lon": aggregated_table_gpd[('Location',"INTPTLON10")].iat[0]},
#     mapbox_style="carto-positron",
#     color_continuous_scale='Portland',
#     opacity=0.25,
#     zoom=10,
#     height=800,
#     width=1000
#   )
# fig.show()

In [ ]:
app = JupyterDash(__name__)

county_df = county_file
county_df["County"] = county_df["County"].str.replace(" ","_")


app.layout = html.Div([
    html.H1("Hazus Demographic Map by Census Block",
            style={
                'text-align':'center',
                'color':'#0868ac',
            }
           ),
    html.Label('Choose a State',
              style = {
                  'font-family':'MathSansBold'
              }
              ),
    dcc.Dropdown(county_df["Abbreviation"].unique(),
                 id = 'state_dropdown', multi = False),
    html.Br(),

    html.Label('Choose a County'),
    dcc.Dropdown(id = 'county_dropdown'),
    html.Br(),
    html.Div(children = '',id = 'index'),

    html.Br(),
    dcc.Markdown(children = '', id = 'path'),
    html.Br(),
    dcc.Markdown(children = '', id = 'path2'),
    html.Label("Choose a table"),
    html.Div([

        dcc.Graph(id = "map")])
])


@app.callback(
    Output(component_id = 'county_dropdown', component_property='options' ),
    Input(component_id = 'state_dropdown', component_property = 'value')
)


def update_dropdown(value):
    df = county_df.loc[county_df["Abbreviation"] == value]["County"]
    return df

@app.callback(
    Output(component_id = 'index', component_property='value' ),
    Input(component_id = 'state_dropdown', component_property = 'value'),
    Input(component_id = 'county_dropdown',component_property = 'value')
)

def update_index(state,county):
    state_code = county_df.loc[county_df["Abbreviation"] == state]
    county_code = state_code.loc[state_code["County"] == county]["FIPS"]
    code = county_code.iat[0]
    return u'{}'.format(
        code
        )

@app.callback(
    Output(component_id = 'path', component_property='value' ),
    Input(component_id = 'state_dropdown', component_property = 'value'),
    Input(component_id = 'county_dropdown',component_property = 'value'),
    Input(component_id = 'index',component_property = 'value')
)

def update_path(state, county, index):
    workdir = os.getcwd() + '/dash_test_data/'
    return u'{}{}/{}_{}_{}.csv'.format(
        workdir,state, state, index, county
    )

@app.callback(
    Output(component_id = 'path2', component_property='value' ),
    Input(component_id = 'state_dropdown', component_property = 'value'),
    Input(component_id = 'index',component_property = 'value')
)

def update_path2(state,index):
    workdir = os.getcwd() + '/dash_test_data/'
    index = index[:2]
    return u'{}{}/tl_2010_{}_tabblock10.zip'.format(
        workdir,state,index
    )


@app.callback(
    Output(component_id = 'map', component_property='figure' ),
    Input(component_id = 'path', component_property = 'value'),
    Input(component_id = 'path2', component_property = 'value'),
)

def update_graph(path,path2):
    df1 = pd.read_csv(path, index_col = 0, header = [0,1])
    df2 = gpd.read_file(path2)
    df2["INTPTLON10"] = df2["INTPTLON10"].astype(float)
    df2["INTPTLAT10"] = df2["INTPTLAT10"].astype(float)
    df2["CensusBlock"] = df2["GEOID10"].astype(int)
    df2 = df2.drop("GEOID10", axis = 1)
    df2 = df2.set_index("CensusBlock")
    df2 = set_multiindex(df2,"Location")
    aggregated_table = df1.merge(df2, how = "inner", left_index = True, right_index = True)
    aggregated_table_gpd = gpd.GeoDataFrame(aggregated_table, geometry =('Location','geometry'))
    mapplot = px.choropleth_mapbox(
    data_frame = aggregated_table_gpd,
    geojson=aggregated_table_gpd[('Location','geometry')],
    locations=aggregated_table_gpd.index,
    center={"lat": aggregated_table_gpd[('Location',"INTPTLAT10")].iat[0], "lon": aggregated_table_gpd[('Location',"INTPTLON10")].iat[0]},
    mapbox_style="carto-positron",
    color_continuous_scale='Portland',
    opacity=0.25,
    zoom=10,
    height=800,
    width=1000
  )
    return (mapplot)
app.run_server(port = 1234)


Dash app running on http://127.0.0.1:1234/
